In [1]:
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
import json
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

In [2]:
from data import *
dataset = semeval2017Dataset(
    dataPath='../dataset/semeval2017-task8/', 
    type='train'
)
glove25d = KeyedVectors.load_word2vec_format(
    '../dataset/glove/glove.twitter.27B.25d.gensim.txt',
    binary=False
)

In [3]:
from torch.utils.data import DataLoader
from data import batchWord2Vec
loader = DataLoader(
    dataset,
    shuffle = True,
    collate_fn = lambda x: batchWord2Vec(x, glove25d)
)

In [7]:
for thread in loader:
    print(thread['stanceTag'].shape)
    print(thread['nodeFeature'].shape)
    break

torch.Size([16])
torch.Size([16, 30, 25])


In [ ]:
from ABGCN import *

In [5]:
# from ABGCN import *
from torch import optim
model = ABGCN(
    w2vDim = 25,
    s2vDim = 64, # 使用的句嵌入的维度
    gcnHiddenDim = 64, # GCN隐藏层的维度（GCNconv1的输出维度）
    rumorFeatureDim = 64, # GCN输出层的维度
    numRumorTag = 3, # 谣言标签种类数
    numStanceTag = 4, # 立场标签种类数
    s2vMethon = 'a', # 获取据嵌入的方法（l:lstm; a:attention）
)
device = torch.device('cuda')
model = model.set_device(device)
loss_func = torch.nn.CrossEntropyLoss(reduction='mean').to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-2, momentum=0.9)
        

In [51]:
from tqdm import tqdm
from torch.nn.functional import softmax

totalLoss = 0.
testdata = dataset[0]
for data in tqdm(iter(loader), 
                 desc="[epoch {:d}, rumor]".format(1),
                 leave=False, 
                 ncols=100):
    # 抹除dataloader生成batch时对数据的升维
    data['threadId'] = data['threadId'][0]
    data['threadIndex'] = data['threadIndex'][0]
    for i in range(len(data['nodeFeature'])):
        data['nodeFeature'][i] = data['nodeFeature'][i].view((data['nodeFeature'][i].shape[1], data['nodeFeature'][i].shape[2]))
    data['edgeIndexTD'] = data['edgeIndexTD'].view((data['edgeIndexTD'].shape[1], data['edgeIndexTD'].shape[2]))
    data['edgeIndexBU'] = data['edgeIndexBU'].view((data['edgeIndexBU'].shape[1], data['edgeIndexBU'].shape[2]))
    data['rumorTag'] = data['rumorTag'].view((data['rumorTag'].shape[1]))
    data['stanceTag'] = data['stanceTag'].view((data['stanceTag'].shape[1]))
    rumorTag = data['rumorTag'].to(device)

    optimizer.zero_grad()
    p = model.forwardRumor(data)
    loss = loss_func(p, rumorTag)
    totalLoss += loss
    loss.backward()
    optimizer.step()

    p = softmax(p, dim=1)


In [52]:
print(totalLoss)

tensor(280.0627, device='cuda:0', grad_fn=<AddBackward0>)


In [48]:
print(loss)

tensor(0.9175, device='cuda:0', grad_fn=<NllLossBackward0>)
